In [1]:
import pandas as pd
import pulp
from pulp import LpProblem, LpMinimize, LpVariable, PULP_CBC_CMD

In [2]:
hubs = pd.read_csv("Hubs-Hubs.csv")
focus_cities = pd.read_csv("Focus Cities-Focus Cities.csv")
demand_with_CountryCity = pd.read_csv("Centeres-Centers.csv")
costs_df = pd.read_csv("Costs.csv")

In [3]:
# Checking types
print (hubs.dtypes)
print (focus_cities.dtypes)

City                       object
Current tons               object
Capacity (monthly tons)    object
dtype: object
City                       object
Airport                    object
Capacity (monthly tons)    object
dtype: object


In [4]:
# checking to see if there are any commas
print(hubs['Capacity (monthly tons)'].unique())
print(focus_cities['Capacity (monthly tons)'].unique())

['95,650' '44,350']
['85,000' '19,000' '36,000']


In [5]:
# Remove commas from the Capacity columns and convert them to numeric
hubs['Capacity (monthly tons)'] = hubs['Capacity (monthly tons)'].replace({',': ''}, regex=True)
focus_cities['Capacity (monthly tons)'] = focus_cities['Capacity (monthly tons)'].replace({',': ''}, regex=True)

# Convert to numeric values (the coercion will handle non-numeric values by setting them as NaN)
hubs['Capacity (monthly tons)'] = pd.to_numeric(hubs['Capacity (monthly tons)'], errors='coerce')
focus_cities['Capacity (monthly tons)'] = pd.to_numeric(focus_cities['Capacity (monthly tons)'], errors='coerce')

In [6]:
# Making sure the types are correct
print (hubs.dtypes)
print (focus_cities.dtypes)

City                       object
Current tons               object
Capacity (monthly tons)     int64
dtype: object
City                       object
Airport                    object
Capacity (monthly tons)     int64
dtype: object


In [7]:
# Creating dfs that only have the infomration we need
hub_supply = hubs[["City", "Capacity (monthly tons)"]]
city_supply = focus_cities[["City", "Capacity (monthly tons)"]]

print (hub_supply.head())
print (city_supply.head())

                                 City  Capacity (monthly tons)
0  Cincinnati/Northern Kentucky (CVG)                    95650
1           Alliance Fort Worth (AFW)                    44350
                                City  Capacity (monthly tons)
0                   Leipzig, Germany                    85000
1                   Hyderabad, India                    19000
2  San Bernardino, CA, United States                    36000


In [8]:
# Now perform a merge
df_supply = pd.merge(hub_supply, city_supply, on= 'City', how='outer')
print (df_supply)

                                 City  Capacity (monthly tons)_x  \
0  Cincinnati/Northern Kentucky (CVG)                    95650.0   
1           Alliance Fort Worth (AFW)                    44350.0   
2                    Leipzig, Germany                        NaN   
3                    Hyderabad, India                        NaN   
4   San Bernardino, CA, United States                        NaN   

   Capacity (monthly tons)_y  
0                        NaN  
1                        NaN  
2                    85000.0  
3                    19000.0  
4                    36000.0  


In [9]:
# Combine the two capacity columns, filling NaN values with 0
df_supply['Capacity (monthly tons)'] = df_supply['Capacity (monthly tons)_x'].fillna(0) + df_supply['Capacity (monthly tons)_y'].fillna(0)
df_supply = df_supply.drop(columns=['Capacity (monthly tons)_x', 'Capacity (monthly tons)_y'])
print (df_supply)

                                 City  Capacity (monthly tons)
0  Cincinnati/Northern Kentucky (CVG)                  95650.0
1           Alliance Fort Worth (AFW)                  44350.0
2                    Leipzig, Germany                  85000.0
3                    Hyderabad, India                  19000.0
4   San Bernardino, CA, United States                  36000.0


In [10]:
demand_with_CountryCity.dtypes

Country                  object
City                     object
Demand (monthly tons)    object
dtype: object

In [11]:
# get rid of potential commas and change demand to a int
demand_with_CountryCity['Demand (monthly tons)'] = demand_with_CountryCity['Demand (monthly tons)'].replace({',': ''}, regex=True)

# Convert to numeric values (the coercion will handle non-numeric values by setting them as NaN)
demand_with_CountryCity['Demand (monthly tons)'] = pd.to_numeric(demand_with_CountryCity['Demand (monthly tons)'], errors='coerce')

demand_with_CountryCity.dtypes

Country                  object
City                     object
Demand (monthly tons)     int64
dtype: object

In [12]:
df_demand = demand_with_CountryCity[["City", "Demand (monthly tons)"]]
df_demand.head()

,City,Demand (monthly tons)
0,Paris,6500
1,Cologne,640
2,Hanover,180
3,Bangalore,9100
4,Coimbatore,570


In [13]:
# Turn supply and demand into a dictionary
supply = dict(zip(df_supply["City"], df_supply["Capacity (monthly tons)"]))
demand = dict(zip(df_demand["City"], df_demand["Demand (monthly tons)"]))

# Testing
print(supply["Cincinnati/Northern Kentucky (CVG)"])  # Should print 95650
print(demand["Paris"])                         # Should print 6500
print(type(supply))                            # Should say <class 'dict'>

95650.0
6500
<class 'dict'>


In [14]:
# We have a valid demand and Supply dictionary, we need to create one for the costs
# Start with checking the types in the costs df
print(costs_df.dtypes)

Cities                                 object
Cincinnati/  Northern Kentucky hub    float64
Alliance   Fort Worth hub             float64
Leipzig   focus city                  float64
Hyderabad focus city                  float64
San Bernadino focus city              float64
dtype: object


In [15]:
# Remove any non-printing characters and extra spaces
costs_df.columns = costs_df.columns.str.replace(r'\s+', '', regex=True)  # Remove all spaces
costs_df.columns = costs_df.columns.str.replace(r'[^\x00-\x7F]+', '', regex=True)  # Remove non-ASCII characters

# To ensure there are no leading/trailing spaces
costs_df.columns = costs_df.columns.str.strip()

costs_df.head(1)

,Cities,Cincinnati/NorthernKentuckyhub,AllianceFortWorthhub,Leipzigfocuscity,Hyderabadfocuscity,SanBernadinofocuscity
0,Paris,1.6,NaN,0.5,1.1,NaN


In [16]:
# I want to rename the column names to make life easier and match the hub names and focus cities with the other dfs
costs_df.rename(columns={
    'Cincinnati/NorthernKentuckyhub': 'CVG',
    'AllianceFortWorthhub': 'AFW',
    'Leipzigfocuscity': 'Leipzig',
}, inplace=True)
costs_df.head()

,Cities,CVG,AFW,Leipzig,Hyderabadfocuscity,SanBernadinofocuscity
0,Paris,1.6,NaN,0.5,1.1,NaN
1,Cologne,1.5,NaN,0.5,1.0,NaN
2,Hanover,1.5,NaN,0.5,1.0,NaN
3,Bengaluru,NaN,NaN,1.5,0.5,NaN
4,Coimbatore,NaN,NaN,1.5,0.5,NaN


In [17]:
costs_df.set_index('Cities', inplace=True)

In [18]:
# Initialize the cost dictionary
cost = {}

# Iterate through the DataFrame
for city in costs_df.index:
    for supplier in costs_df.columns:
        cost_var = costs_df.loc[city, supplier]
        if pd.notna(cost_var):  # Skip NaN values
            cost[(supplier, city)] = cost_var

# Display the resulting cost dictionary
print(cost)

{('CVG', 'Paris'): 1.6, ('Leipzig', 'Paris'): 0.5, ('Hyderabadfocuscity', 'Paris'): 1.1, ('CVG', 'Cologne'): 1.5, ('Leipzig', 'Cologne'): 0.5, ('Hyderabadfocuscity', 'Cologne'): 1.0, ('CVG', 'Hanover'): 1.5, ('Leipzig', 'Hanover'): 0.5, ('Hyderabadfocuscity', 'Hanover'): 1.0, ('Leipzig', 'Bengaluru'): 1.5, ('Hyderabadfocuscity', 'Bengaluru'): 0.5, ('Leipzig', 'Coimbatore'): 1.5, ('Hyderabadfocuscity', 'Coimbatore'): 0.5, ('Leipzig', 'Delhi'): 1.5, ('Hyderabadfocuscity', 'Delhi'): 0.5, ('Leipzig', 'Mumbai'): 1.5, ('Hyderabadfocuscity', 'Mumbai'): 0.5, ('CVG', 'Cagliari'): 1.5, ('Leipzig', 'Cagliari'): 0.5, ('Hyderabadfocuscity', 'Cagliari'): 1.0, ('CVG', 'Catania'): 1.5, ('Leipzig', 'Catania'): 0.5, ('Hyderabadfocuscity', 'Catania'): 1.0, ('CVG', 'Milan'): 1.5, ('Leipzig', 'Milan'): 0.5, ('Hyderabadfocuscity', 'Milan'): 1.0, ('CVG', 'Rome'): 1.5, ('Leipzig', 'Rome'): 0.5, ('Hyderabadfocuscity', 'Rome'): 1.1, ('CVG', 'Katowice'): 1.4, ('Leipzig', 'Katowice'): 0.5, ('Hyderabadfocuscity', 

In [19]:
# Initialize the Problem
prob = pulp.LpProblem("Amazon_Air_Minimize_Costs", pulp.LpMinimize)

In [20]:
# Define Variables
# Decision variables: how much to ship from each supplier to each center
x = pulp.LpVariable.dicts("ship", cost.keys(), lowBound=0, cat="Continuous")

In [21]:
# Objective Function
prob += pulp.lpSum([cost[(i, j)] * x[(i, j)] for (i, j) in cost]), "Total_Shipping_Cost"

In [22]:
total_supply = sum(supply.values())
total_demand = sum(demand.values())

In [23]:
# Smaller epsilon margin
epsilon = 0.00 * total_supply  

# Scaled supply
scaling_factor = 0.1
scaled_supply = {i: supply[i] * scaling_factor for i in supply}
scaled_demand = {j: demand[j] * scaling_factor for j in demand}

# Adjust constraints accordingly

# Supply constraints
for i in scaled_supply:
    prob += pulp.lpSum([x[(i, j)] for (i, j) in cost if i == i]) <= scaled_supply[i] + epsilon, f"Supply_Cap_{i}"

# Demand constraints
for j in scaled_demand:
    prob += pulp.lpSum([x[(i, j)] for (i, j) in cost if j == j]) >= scaled_demand[j] - epsilon, f"Demand_{j}"

In [24]:
# Solve
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.11/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/cn/5rls9kg91b35jqphwksc14700000gn/T/ee0463ba814d4753b606053d40fb95d8-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/cn/5rls9kg91b35jqphwksc14700000gn/T/ee0463ba814d4753b606053d40fb95d8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 75 COLUMNS
At line 13708 RHS
At line 13779 BOUNDS
At line 13780 ENDATA
Problem MODEL has 70 rows, 192 columns and 13440 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-70) rows, 0 (-192) columns and 0 (-13440) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 475
After Postsolve, objective 475, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 475 - 0 iterations time 0.002, Presolve 0.00
Option for printingOp

1

In [25]:
# Results
print("Status:", pulp.LpStatus[prob.status])
print(f"Total Minimized Cost: ${pulp.value(prob.objective):,.2f}\n")

print("Shipping Plan:")
for (i, j) in x:
    if x[(i, j)].varValue > 0:
        print(f"{i} -> {j}: {x[(i, j)].varValue:.2f} tons")

# Okay, So. I need that status to be optimal, as i am trying to optimize things
# It technically runs correctly, threre is no error
# The problem is that when i go to check how much is shipped (shown a bit below) its 0 when it should NOT be
# I cant figure our why its shipping nothing, there is plenty of supply for the demand and even if that wasnt the case 

Status: Optimal
Total Minimized Cost: $475.00

Shipping Plan:
AFW -> Austin: 1900.00 tons
